# 🧹 Template de Limpieza de Datos - Capa Bronce

## Objetivo
Cargar datos desde MinIO (capa Bronce) y realizar operaciones de limpieza y transformación en un DataFrame.

In [116]:
# Importaciones necesarias
import os
import sys
import pandas as pd
from pyspark.sql import SparkSession
from minio import Minio
from io import BytesIO
import tempfile
import warnings
warnings.filterwarnings('ignore')

In [117]:
# 🔧 Configuración de MinIO
# Obtener credenciales desde variables de entorno (O configurarlas directamente aquí)

MINIO_ENDPOINT = os.environ.get('MINIO_ENDPOINT', 'localhost:9000')
MINIO_ACCESS_KEY = os.environ.get('MINIO_ACCESS_KEY', 'minioadmin')
MINIO_SECRET_KEY = os.environ.get('MINIO_SECRET_KEY', 'minioadmin')
MINIO_BUCKET = os.environ.get('MINIO_BUCKET', 'meteo-bronze')

print(f"✅ Conectando a MinIO: {MINIO_ENDPOINT}")
print(f"📦 Bucket: {MINIO_BUCKET}")

# Crear cliente MinIO
minio_client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)

print("✅ Conexión a MinIO establecida")

✅ Conectando a MinIO: localhost:9000
📦 Bucket: meteo-bronze
✅ Conexión a MinIO establecida


In [118]:
# ⚡ Inicializar SparkSession

spark = SparkSession.builder \
    .appName("LimpiezaDatos") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("✅ SparkSession iniciada")
print("   df = PySpark DataFrame")
print("   Métodos: .show(), .filter(), .select(), .rename(), etc.")

✅ SparkSession iniciada
   df = PySpark DataFrame
   Métodos: .show(), .filter(), .select(), .rename(), etc.


In [119]:
# 🔄 Funciones para cargar archivos CSV desde MinIO a PySpark

def cargar_csv_desde_minio(nombre_archivo):
    """
    Carga un archivo CSV desde MinIO a un PySpark DataFrame.
    
    CSV no tiene problemas de nanosegundos (timestamps como strings).
    
    Args:
        nombre_archivo (str): Nombre/ruta del archivo en MinIO
        
    Returns:
        pyspark.sql.DataFrame: DataFrame de PySpark con los datos del archivo
    """
    try:
        print(f"📥 Descargando desde MinIO: {nombre_archivo}")
        
        # Descargar archivo a archivo temporal
        temp_dir = tempfile.gettempdir()
        temp_file = os.path.join(temp_dir, nombre_archivo.split('/')[-1])
        
        minio_client.fget_object(MINIO_BUCKET, nombre_archivo, temp_file)
        print(f"   ✅ Descargado a: {temp_file}")
        
        # Leer CSV con PySpark (inferir esquema automáticamente)
        df = spark.read.csv(temp_file, header=True, inferSchema=True)
        
        row_count = df.count()
        col_count = len(df.columns)
        print(f"✅ Cargado en PySpark: {row_count} filas, {col_count} columnas")
        
        return df
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None


def cargar_csv_reciente(nombre_tabla):
    """
    Carga el archivo CSV más reciente de una tabla desde MinIO a PySpark.
    
    Busca archivos con patrón: {tabla}_bronce_{timestamp}.csv
    
    Args:
        nombre_tabla (str): Nombre de la tabla
        
    Returns:
        pyspark.sql.DataFrame: DataFrame de PySpark con los datos
    """
    try:
        # Listar archivos de la tabla
        objects = minio_client.list_objects(MINIO_BUCKET, prefix=nombre_tabla, recursive=True)
        
        archivos = []
        for obj in objects:
            if obj.object_name.endswith('.csv') and '_bronce_' in obj.object_name:
                archivos.append(obj.object_name)
        
        if not archivos:
            print(f"⚠️  No hay archivos CSV para la tabla: {nombre_tabla}")
            return None
        
        # Ordenar por nombre (el timestamp está en el nombre) y tomar el más reciente
        archivo_reciente = sorted(archivos)[-1]
        print(f"📄 Archivo más reciente: {archivo_reciente}")
        
        return cargar_csv_desde_minio(archivo_reciente)
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

In [120]:
# 📥 CARGAR DATOS COMO PYSPARK DATAFRAME DESDE CSV

# Cargar el archivo CSV más reciente de sensor_readings
df = cargar_csv_reciente("sensor_readings")

📄 Archivo más reciente: sensor_readings/sensor_readings_bronce_20251202112554.csv
📥 Descargando desde MinIO: sensor_readings/sensor_readings_bronce_20251202112554.csv
   ✅ Descargado a: C:\Users\ALUMNO~1\AppData\Local\Temp\sensor_readings_bronce_20251202112554.csv
✅ Cargado en PySpark: 97 filas, 12 columnas
✅ Cargado en PySpark: 97 filas, 12 columnas


In [121]:
# 🔍 INSPECCIONAR PYSPARK DATAFRAME

if df is not None:
    print("=" * 80)
    print("📊 INFORMACIÓN DEL DATAFRAME")
    print("=" * 80)
    
    print(f"\n📏 Dimensiones:")
    print(f"   Filas: {df.count()}")
    print(f"   Columnas: {len(df.columns)}")
    
    print(f"\n🔤 Esquema:")
    df.printSchema()
    
    print(f"\n👀 Primeras 5 filas:")
    df.show(5)
else:
    print("⚠️  No hay datos cargados")

📊 INFORMACIÓN DEL DATAFRAME

📏 Dimensiones:
   Filas: 97
   Columnas: 12

🔤 Esquema:
root
 |-- id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- ip: string (nullable = true)
 |-- temperature: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- pm25: double (nullable = true)
 |-- light: integer (nullable = true)
 |-- uv_level: integer (nullable = true)
 |-- pressure: double (nullable = true)
 |-- rain_raw: integer (nullable = true)
 |-- wind_raw: integer (nullable = true)
 |-- vibration: boolean (nullable = true)


👀 Primeras 5 filas:
+---+--------------------+------------+-----------+--------+----+-----+--------+--------+--------+--------+---------+
| id|           timestamp|          ip|temperature|humidity|pm25|light|uv_level|pressure|rain_raw|wind_raw|vibration|
+---+--------------------+------------+-----------+--------+----+-----+--------+--------+--------+--------+---------+
|  1|2025-10-23 14:00:...|10.207.51.79|       25.0|    39.0

---

## 🧹 Limpieza de Datos con PySpark

Usa las celdas siguientes para realizar tu limpieza personalizada.

In [122]:
print("=" * 80)
print("📊 PYSPARK DATAFRAME COMPLETO")
print("=" * 80)
df.show()

📊 PYSPARK DATAFRAME COMPLETO
+---+--------------------+------------+-----------+--------+----+-----+--------+--------+--------+--------+---------+
| id|           timestamp|          ip|temperature|humidity|pm25|light|uv_level|pressure|rain_raw|wind_raw|vibration|
+---+--------------------+------------+-----------+--------+----+-----+--------+--------+--------+--------+---------+
|  1|2025-10-23 14:00:...|10.207.51.79|       25.0|    39.0| 5.0| 2426|       0|   939.0|       0|       0|     true|
|  2|2025-10-23 14:00:...|10.207.51.79|       25.0|    39.0| 5.0| 2422|       0|   939.0|       0|       0|     true|
|  3|2025-10-23 14:00:...|10.207.51.79|       25.0|    40.0| 5.0| 2371|       0|   939.0|       0|       0|     true|
|  4|2025-10-23 14:01:...|10.207.51.79|       25.0|    40.0| 5.0| 2382|       0|   939.0|       0|       0|     true|
|  5|2025-10-23 14:01:...|10.207.51.79|       25.0|    40.0| 5.0| 2410|       0|   939.0|       0|       0|     true|
|  6|2025-10-23 14:01:...|1

In [123]:
# Mostrar primeras 10 filas con PySpark
df.show(10)

+---+--------------------+------------+-----------+--------+----+-----+--------+--------+--------+--------+---------+
| id|           timestamp|          ip|temperature|humidity|pm25|light|uv_level|pressure|rain_raw|wind_raw|vibration|
+---+--------------------+------------+-----------+--------+----+-----+--------+--------+--------+--------+---------+
|  1|2025-10-23 14:00:...|10.207.51.79|       25.0|    39.0| 5.0| 2426|       0|   939.0|       0|       0|     true|
|  2|2025-10-23 14:00:...|10.207.51.79|       25.0|    39.0| 5.0| 2422|       0|   939.0|       0|       0|     true|
|  3|2025-10-23 14:00:...|10.207.51.79|       25.0|    40.0| 5.0| 2371|       0|   939.0|       0|       0|     true|
|  4|2025-10-23 14:01:...|10.207.51.79|       25.0|    40.0| 5.0| 2382|       0|   939.0|       0|       0|     true|
|  5|2025-10-23 14:01:...|10.207.51.79|       25.0|    40.0| 5.0| 2410|       0|   939.0|       0|       0|     true|
|  6|2025-10-23 14:01:...|10.207.51.79|       25.0|    4

In [124]:
# Información del PySpark DataFrame
print(f"Filas: {df.count()}")
print(f"Columnas: {len(df.columns)}")
print(f"\nEsquema:")
df.printSchema()

Filas: 97
Columnas: 12

Esquema:
root
 |-- id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- ip: string (nullable = true)
 |-- temperature: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- pm25: double (nullable = true)
 |-- light: integer (nullable = true)
 |-- uv_level: integer (nullable = true)
 |-- pressure: double (nullable = true)
 |-- rain_raw: integer (nullable = true)
 |-- wind_raw: integer (nullable = true)
 |-- vibration: boolean (nullable = true)



In [125]:
# 🧹 EJEMPLO 1: Filtrar por rango de valores (PySpark SQL)

df_clean = df.filter("temperature >= 10 AND temperature <= 40")
print(f"✅ Filas en rango válido: {df_clean.count()}")
df_clean.show(5)

✅ Filas en rango válido: 97
+---+--------------------+------------+-----------+--------+----+-----+--------+--------+--------+--------+---------+
| id|           timestamp|          ip|temperature|humidity|pm25|light|uv_level|pressure|rain_raw|wind_raw|vibration|
+---+--------------------+------------+-----------+--------+----+-----+--------+--------+--------+--------+---------+
|  1|2025-10-23 14:00:...|10.207.51.79|       25.0|    39.0| 5.0| 2426|       0|   939.0|       0|       0|     true|
|  2|2025-10-23 14:00:...|10.207.51.79|       25.0|    39.0| 5.0| 2422|       0|   939.0|       0|       0|     true|
|  3|2025-10-23 14:00:...|10.207.51.79|       25.0|    40.0| 5.0| 2371|       0|   939.0|       0|       0|     true|
|  4|2025-10-23 14:01:...|10.207.51.79|       25.0|    40.0| 5.0| 2382|       0|   939.0|       0|       0|     true|
|  5|2025-10-23 14:01:...|10.207.51.79|       25.0|    40.0| 5.0| 2410|       0|   939.0|       0|       0|     true|
+---+--------------------+--

In [126]:
# 🧹 EJEMPLO 2: Convertir tipos de datos (PySpark cast)

from pyspark.sql.functions import col

df_clean = df.select(
    col("temperature").cast("float").alias("temperature"),
    col("humidity").cast("float").alias("humidity"),
    "*"
)
print(f"✅ Tipos convertidos:")
df_clean.printSchema()

✅ Tipos convertidos:
root
 |-- temperature: float (nullable = true)
 |-- humidity: float (nullable = true)
 |-- id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- ip: string (nullable = true)
 |-- temperature: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- pm25: double (nullable = true)
 |-- light: integer (nullable = true)
 |-- uv_level: integer (nullable = true)
 |-- pressure: double (nullable = true)
 |-- rain_raw: integer (nullable = true)
 |-- wind_raw: integer (nullable = true)
 |-- vibration: boolean (nullable = true)



In [127]:
# .count() devuelve el número total de filas del DataFrame
row_count = df.count()
print(f'Number of rows: {row_count}')

Number of rows: 97


In [128]:
# 🧹 EJEMPLO 3: Eliminar duplicados (PySpark distinct)

df_clean = df.distinct()
print(f"✅ Filas después de eliminar duplicados: {df_clean.count()}")
df_clean.show(5)

✅ Filas después de eliminar duplicados: 97
+---+--------------------+------------+-----------+--------+----+-----+--------+--------+--------+--------+---------+
| id|           timestamp|          ip|temperature|humidity|pm25|light|uv_level|pressure|rain_raw|wind_raw|vibration|
+---+--------------------+------------+-----------+--------+----+-----+--------+--------+--------+--------+---------+
| 78|2025-10-23 14:09:...|10.207.51.79|       24.0|    38.0| 5.0| 2373|       0|   939.0|       0|       0|     true|
| 22|2025-10-23 14:02:...|10.207.51.79|       24.0|    39.0| 6.0| 2452|       0|   939.0|       0|       0|     true|
| 86|2025-10-23 14:10:...|10.207.51.79|       24.0|    39.0| 5.0| 2371|       0|   939.0|       0|       0|     true|
| 38|2025-10-23 14:04:...|10.207.51.79|       24.0|    39.0| 5.0| 2422|       0|   939.0|       0|       0|     true|
| 54|2025-10-23 14:05:...|10.207.51.79|       24.0|    39.0| 5.0| 2466|       0|   939.0|       0|       0|     true|
+---+--------

In [129]:
# 🧹 EJEMPLO 4: Seleccionar y renombrar columnas (PySpark)

df_clean = df.select(
    col("temperature").alias("temp"),
    col("humidity").alias("humedad"),
    "timestamp"
)
print(f"✅ Columnas seleccionadas: {df_clean.columns}")
df_clean.show(5)

✅ Columnas seleccionadas: ['temp', 'humedad', 'timestamp']
+----+-------+--------------------+
|temp|humedad|           timestamp|
+----+-------+--------------------+
|25.0|   39.0|2025-10-23 14:00:...|
|25.0|   39.0|2025-10-23 14:00:...|
|25.0|   40.0|2025-10-23 14:00:...|
|25.0|   40.0|2025-10-23 14:01:...|
|25.0|   40.0|2025-10-23 14:01:...|
+----+-------+--------------------+
only showing top 5 rows



In [130]:
df_clean = df_clean.drop("uv_level")
df_clean = df_clean.drop("vibration")
df_clean = df_clean.drop("rain_raw")
df_clean = df_clean.drop("wind_raw")
df_clean = df_clean.drop("pressure")
df_clean.show()

+----+-------+--------------------+
|temp|humedad|           timestamp|
+----+-------+--------------------+
|25.0|   39.0|2025-10-23 14:00:...|
|25.0|   39.0|2025-10-23 14:00:...|
|25.0|   40.0|2025-10-23 14:00:...|
|25.0|   40.0|2025-10-23 14:01:...|
|25.0|   40.0|2025-10-23 14:01:...|
|25.0|   40.0|2025-10-23 14:01:...|
|25.0|   40.0|2025-10-23 14:01:...|
|25.0|   40.0|2025-10-23 14:01:...|
|25.0|   40.0|2025-10-23 14:01:...|
|25.0|   39.0|2025-10-23 14:01:...|
|25.0|   38.0|2025-10-23 14:01:...|
|25.0|   38.0|2025-10-23 14:01:...|
|25.0|   37.0|2025-10-23 14:01:...|
|24.0|   38.0|2025-10-23 14:02:...|
|24.0|   38.0|2025-10-23 14:02:...|
|24.0|   38.0|2025-10-23 14:02:...|
|24.0|   39.0|2025-10-23 14:02:...|
|24.0|   39.0|2025-10-23 14:02:...|
|24.0|   39.0|2025-10-23 14:02:...|
|24.0|   39.0|2025-10-23 14:02:...|
+----+-------+--------------------+
only showing top 20 rows



1. Significado de las Columnas
id: Es un identificador único para cada registro.

Observación: Fíjate que no están en orden (78, 22, 86...). Esto es normal en Spark porque procesa los datos en paralelo y no garantiza el orden visual a menos que lo ordenes explícitamente (orderBy).

timestamp: La fecha y hora exacta en que se tomó la lectura.

Dato: Las lecturas son del 23 de Octubre de 2025, entre las 14:00 y 14:10 horas.

ip: La dirección IP del dispositivo que envió los datos (10.207.51.79).

Dato: Al ser la misma en todas las filas, indica que todos estos datos provienen de una sola estación o sensor.

temperature: La temperatura ambiente en grados Celsius (probablemente).

Valor: Oscila entre 24.0°C y 25.0°C.

humidity: La Humedad Relativa en porcentaje (%).

Valor: Oscila entre 38% y 40%. Es un ambiente seco/confortable.

pm25: Partículas por Millón de 2.5 micras (PM2.5). Es un indicador de calidad del aire.

Valor: 5.0 - 6.0. Son valores bajos, lo que indica una buena calidad del aire.

light: Nivel de luminosidad o intensidad de luz (probablemente en Lux o un valor crudo del sensor analógico).

Valor: Alrededor de 2400. Indica que es de día o hay luz artificial fuerte.

In [ ]:
## 🏆 CAPA SILVER: Guardar Datos Limpios en MinIO

Una vez que hayas limpiado los datos, puedes guardarlos en la capa Silver de MinIO.

La capa Silver almacena datos procesados y limpios en formato Parquet.

**Buckets:**
- **Bronce** (`meteo-bronze`): Datos crudos del ETL (CSV)
- **Silver** (`meteo-silver`): Datos limpios y transformados (Parquet)

In [131]:
# 💾 GUARDAR DATOS LIMPIOS EN CAPA SILVER (MinIO)

import os
from datetime import datetime

# Crear bucket Silver si no existe
bucket_silver = MINIO_BUCKET.replace("-bronze", "-silver")

try:
    if not minio_client.bucket_exists(bucket_silver):
        minio_client.make_bucket(bucket_silver)
        print(f"[OK] Bucket Silver creado: {bucket_silver}")
    else:
        print(f"[OK] Bucket Silver existe: {bucket_silver}")
except Exception as e:
    print(f"[ERROR] Error creando bucket: {e}")

# Guardar el DataFrame limpiado en formato Parquet
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
archivo_silver = f"sensor_readings_silver_{timestamp}.parquet"

temp_dir = tempfile.gettempdir()
temp_file = os.path.join(temp_dir, archivo_silver)

print(f"\n[INFO] Guardando datos limpios en Parquet...")
print(f"   Archivo: {archivo_silver}")
print(f"   Filas: {df_sin_outliers_temp.count()}")

try:
    # Guardar como Parquet
    df_sin_outliers_temp.write.mode("overwrite").parquet(temp_file)
    
    # Subir a MinIO
    minio_client.fput_object(bucket_silver, archivo_silver, temp_file)
    
    file_size_mb = os.path.getsize(temp_file) / (1024 * 1024)
    print(f"[OK] Archivo guardado en {bucket_silver}")
    print(f"   Tamaño: {file_size_mb:.2f} MB")
    print(f"   Ruta: {bucket_silver}/{archivo_silver}")
    
    # Limpiar temporal
    try:
        os.remove(temp_file)
    except:
        pass
        
except Exception as e:
    print(f"[ERROR] Error guardando en Silver: {e}")
    import traceback
    traceback.print_exc()

[OK] Bucket Silver creado: meteo-silver

[INFO] Guardando datos limpios en Parquet...
   Archivo: sensor_readings_silver_20251202125301.parquet
   Filas: 84
[ERROR] Error guardando en Silver: An error occurred while calling o446.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java

Traceback (most recent call last):
  File "C:\Users\Alumno_AI\AppData\Local\Temp\ipykernel_15916\217100020.py", line 31, in <module>
    df_sin_outliers_temp.write.mode("overwrite").parquet(temp_file)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
  File "c:\Users\Alumno_AI\Desktop\Estacion_Meteorologica\venv_meteo\Lib\site-packages\pyspark\sql\readwriter.py", line 1721, in parquet
  File "c:\Users\Alumno_AI\Desktop\Estacion_Meteorologica\venv_meteo\Lib\site-packages\py4j\java_gateway.py", line 1322, in __call__
    args_command = "".join(
        [get_command_part(arg, self.pool) for arg in new_args])
  File "c:\Users\Alumno_AI\Desktop\Estacion_Meteorologica\venv_meteo\Lib\site-packages\pyspark\errors\exceptions\captured.py", line 179, in deco
  File "c:\Users\Alumno_AI\Desktop\Estacion_Meteorologica\venv_meteo\Lib\site-packages\py4j\protocol.py", line 326, in get_return_value
    if answer[1] == REFERENCE_TYPE:
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^
        raise Py